In [3]:
from pymongo import MongoClient
from gridfs import GridFS

# Connect to MongoDB
db = MongoClient('mongodb://10.49.0.6:27017')['supercias']
companies = db['companies']

#find one
comp = companies.find_one()
#print(comp['Documentos online'])
pdfs = []
#if company has documents
for value in comp['Documentos online'].values():
    if isinstance(value, list) and len(value) > 0:
        [ pdfs.append(v) for v in value ]

print(pdfs)



['1000000000008_DocumentosJuridicos_CANCELACIÓN_Resolución_2007-01-03.pdf']


In [12]:
# Access the GridFS collection
fs = GridFS(db, collection='companies')

# Retrieve all files from GridFS collection that have in the filen ethe substring 'Balance  Estado de Situación'
files = fs.find({'filename': {'$regex': '.*Balance  Estado de Situación Financiera_2021.*'}})

# print how many files were found
# this will exahust the cursor
#print('found:', len(list(files)))




In [18]:
# Write the contents of the file to a new file
counter = 0
filename = ''
filenames = []
for file in files:
    filename = file.filename
    filenames.append(filename)
    if counter >= 100:
        break
    
    with open(filename, 'wb') as f:
        f.write(file.read())
    
    counter += 1

extract the text from the pdf

In [20]:
from pdf2image import convert_from_path

pdfs_images = []
# for every filename pdf
for filename in filenames:
    try:
        # Convert PDF to images
        images = convert_from_path(filename)

        # Save each image as a separate file
        for i, image in enumerate(images):
            pdfs_images.append(f'{filename}_{i}.jpg')
            image.save(f'{filename}_{i}.jpg', 'JPEG')
    except Exception as e:
        print(e)
        print(f'Error processing {filename}')
        continue




Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error (2): Illegal character <48> in hex string
Syntax Error (3): Illegal character <54> in hex string
Syntax Error (4): Illegal character <4d> in hex string
Syntax Error (5): Illegal character <4c> in hex string
Syntax Error (11): Illegal character <48> in hex string
Syntax Error (20): Illegal character <73> in hex string
Syntax Error (23): Illegal character <68> in hex string
Syntax Error (24): Illegal character <72> in hex string
Syntax Error (27): Illegal character <3d> in hex string
Syntax Error (28): Illegal character <22> in hex string
Syntax Error (29): Illegal character <2f> in hex string
Syntax Error (30): Illegal character <22> in hex string
Syntax Error (32): Illegal character <2f> in hex string
Syntax Error (36): Illegal character <73> in hex string
Syntax Error (38): Illegal character <72> in hex string
Syntax Error (39): Illegal character <69> in hex string
Syntax Error (40): Illeg

In [27]:
import easyocr

%pip install opencv-python-headless
reader = easyocr.Reader(['es'])
extract_info = reader.readtext('/home/terac/data-mining/supercias_second/src/clean/output_image_0.jpg')



Defaulting to user installation because normal site-packages is not writeable


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Note: you may need to restart the kernel to use updated packages.


In [28]:
extract_info = [ el[1] for el in extract_info ]
print(extract_info)

['REPUBLICA', 'DEL', 'ECUADOR', 'SUPERINTENDENCIA', 'DE COMPAÑIAS', '8 5', 'RESOLUCION No. 07.Q.IJ.000025', 'DRA. PIEDAD MONCAYO DE VASCONEZ', 'DIRECTORA', 'JURIDICA DEL DEPARTAMENTO', 'DE DISOLUCION Y LIQUIDACION DE COMPAÑIAS', 'co NSIDERA ND O:', 'QUE en virtud de lo dispuesto en el inciso segundo del artículo 405, de la', 'de', 'Compañías; el Superintendente de Compañías; sin ningún otro trámite', 'ordenar la', 'cancelación de la inscripción en el Registro Mercantil de las compañías cuyo trámite de', 'liquidación', 'no', 'hubiere  terminado', 'en', 'el', 'de', 'un', 'año  posterior', 'a', 'la fecha de', 'expedición de la Resolución de Disolución;', 'QUE por Resolución No.85-1-2-1-379 de 21 de', 'marzo de 1985, inscrita en el', 'Registro Mercantil del Distrito Metropolitano de Quito', 'el 6 de marzo', 'de 1986,', 'esta', 'Superintendencia', 'declaró', 'disuelta', 'a', 'la COMPAÑIA', 'AGRICOLA', 'Y PRODUCTORA', 'DE ALIMENTOS BALANCEADOS SOCIEDAD ANONIMA. (CAPRODAL SA), EN', 'LIQUIDACI